In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy.optimize as opt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import svm

from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, cross_val_score

In [2]:
# Training data
train = pd.read_csv("/home/kito/Desktop/sem5/მანქანური სწავლება/blu/train.csv")
# Test data
test = pd.read_csv("/home/kito/Desktop/sem5/მანქანური სწავლება/blu/test.csv")

train.head()

,id,target,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
0,0,1.0,-1.067,-1.114,-0.616,0.376,1.090,0.467,-0.422,0.460,...,0.220,-0.339,0.254,-0.179,0.352,0.125,0.347,0.436,0.958,-0.824
1,1,0.0,-0.831,0.271,1.716,1.096,1.731,-0.197,1.904,-0.265,...,-0.765,-0.735,-1.158,2.554,0.856,-1.506,0.462,-0.029,-1.932,-0.343
2,2,0.0,0.099,1.390,-0.732,-1.065,0.005,-0.081,-1.450,0.317,...,-1.311,0.799,-1.001,1.544,0.575,-0.309,-0.339,-0.148,-0.646,0.725
3,3,1.0,-0.989,-0.916,-1.343,0.145,0.543,0.636,1.127,0.189,...,-1.370,1.093,0.596,-0.589,-0.649,-0.163,-0.958,-1.081,0.805,3.401
4,4,0.0,0.811,-1.509,0.522,-0.360,-0.220,-0.959,0.334,-0.566,...,-0.178,0.718,-1.017,1.249,-0.596,-0.445,1.751,1.442,-0.393,-0.643


In [3]:
train.describe()

,id,target,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
count,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,...,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000
mean,124.500000,0.268000,-0.098064,0.001208,0.090680,-0.122248,0.011500,-0.116624,0.006932,0.100988,...,0.013052,0.007500,0.000452,0.060276,-0.090308,-0.040728,-0.002132,-0.012540,-0.039904,0.073236
std,72.312977,0.443806,0.996063,0.955117,0.968065,0.933001,0.945662,1.081705,1.014091,1.028042,...,1.027845,1.048169,1.026398,1.048744,1.008657,1.051273,1.065524,0.958744,0.948251,0.924989
min,0.000000,0.000000,-3.181000,-3.041000,-2.967000,-2.898000,-2.837000,-3.831000,-2.873000,-2.489000,...,-2.824000,-2.971000,-3.592000,-3.071000,-2.621000,-3.013000,-3.275000,-2.665000,-3.006000,-2.471000
25%,62.250000,0.000000,-0.756250,-0.624750,-0.515750,-0.695500,-0.678000,-0.758500,-0.646250,-0.589000,...,-0.764000,-0.729250,-0.699750,-0.589000,-0.701000,-0.696750,-0.555750,-0.677500,-0.719750,-0.559000
50%,124.500000,0.000000,-0.064500,-0.008000,0.067500,-0.090000,0.028000,-0.073500,-0.076500,0.104500,...,0.017500,0.053000,0.029500,0.042500,-0.102000,-0.057000,0.094500,-0.025500,-0.027500,0.160500
75%,186.750000,1.000000,0.647750,0.493250,0.716000,0.436250,0.625250,0.554250,0.676500,0.717000,...,0.690500,0.665000,0.670750,0.713250,0.532250,0.727000,0.590000,0.671000,0.717000,0.711000
max,249.000000,1.000000,2.347000,3.138000,2.609000,2.590000,2.413000,2.687000,2.793000,3.766000,...,2.773000,2.701000,3.193000,4.280000,2.716000,3.074000,2.626000,2.388000,2.730000,3.401000


In [4]:
# Separate features and target variable
train_x = train.drop(['id', 'target'], axis=1)
train_y = train['target']
test_x = test.drop(['id'], axis=1)
train_x

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-1.067,-1.114,-0.616,0.376,1.090,0.467,-0.422,0.460,-0.443,-0.338,...,0.220,-0.339,0.254,-0.179,0.352,0.125,0.347,0.436,0.958,-0.824
1,-0.831,0.271,1.716,1.096,1.731,-0.197,1.904,-0.265,0.557,1.202,...,-0.765,-0.735,-1.158,2.554,0.856,-1.506,0.462,-0.029,-1.932,-0.343
2,0.099,1.390,-0.732,-1.065,0.005,-0.081,-1.450,0.317,-0.624,-0.017,...,-1.311,0.799,-1.001,1.544,0.575,-0.309,-0.339,-0.148,-0.646,0.725
3,-0.989,-0.916,-1.343,0.145,0.543,0.636,1.127,0.189,-0.118,-0.638,...,-1.370,1.093,0.596,-0.589,-0.649,-0.163,-0.958,-1.081,0.805,3.401
4,0.811,-1.509,0.522,-0.360,-0.220,-0.959,0.334,-0.566,-0.656,-0.499,...,-0.178,0.718,-1.017,1.249,-0.596,-0.445,1.751,1.442,-0.393,-0.643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,-0.068,-0.184,-1.153,0.610,0.414,1.557,-0.234,0.950,0.896,1.416,...,1.492,1.430,-0.333,-0.200,-1.073,0.797,1.980,1.191,1.032,-0.402
246,-0.234,-1.373,-2.050,-0.408,-0.255,0.784,0.986,-0.891,-0.268,-0.569,...,-0.996,0.678,1.395,0.714,0.215,-0.537,-1.267,-1.021,0.747,0.128
247,-2.327,-1.834,-0.762,0.660,-0.858,-2.764,-0.539,-0.065,0.549,1.474,...,-1.237,-0.620,0.670,-2.010,0.438,1.972,-0.379,0.676,-1.220,-0.855
248,-0.451,-0.204,-0.762,0.261,0.022,-1.487,-1.122,0.141,0.369,-0.173,...,0.729,0.411,2.366,-0.021,0.160,0.045,0.208,-2.117,-0.546,-0.093


In [5]:
# Scale features by normalization to avoid bias towards columns
def scaleFeature(dataFrame, column):
    stdVal = np.nanstd(dataFrame[column])
    meanVal = np.mean(dataFrame[column])
    dataFrame[column] = (dataFrame[column] - meanVal)/stdVal

In [6]:
# Scale features for better accuracy
for col in test_x.columns:
    scaleFeature(test_x, col)
for col in train_x.columns:
    scaleFeature(train_x, col)
train_x

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.974717,-1.169956,-0.731456,0.535098,1.142759,0.540623,-0.423820,0.349920,-0.446382,-0.342796,...,0.201746,-0.331240,0.247523,-0.228612,0.439391,0.157961,0.328319,0.468780,1.054473,-0.971943
1,-0.737309,0.283037,1.682304,1.308350,1.821951,-0.074454,1.874460,-0.356719,0.675068,1.183966,...,-0.758492,-0.709799,-1.130922,2.382589,0.940068,-1.396604,0.436464,-0.017203,-1.999355,-0.450893
2,0.198240,1.456971,-0.851523,-1.012478,-0.006887,0.032999,-1.439569,0.210541,-0.649364,-0.024555,...,-1.290767,0.756641,-0.977653,1.417600,0.660921,-0.255700,-0.316787,-0.141573,-0.640454,0.706032
3,-0.896252,-0.962236,-1.483945,0.287014,0.563168,0.697171,1.106720,0.085783,-0.081911,-0.640217,...,-1.348283,1.037693,0.581395,-0.620340,-0.555008,-0.116542,-0.898886,-1.116673,0.892800,3.604844
4,0.914488,-1.584348,0.446442,-0.255336,-0.245293,-0.780311,0.323170,-0.650096,-0.685251,-0.502412,...,-0.186249,0.679209,-0.993273,1.135747,-0.502358,-0.385326,1.648624,1.520175,-0.373112,-0.775872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.030243,-0.194300,-1.287284,0.786405,0.426482,1.550313,-0.238061,0.827510,1.055240,1.396126,...,1.441769,1.359851,-0.325528,-0.248677,-0.976212,0.798469,1.863972,1.257849,1.132668,-0.514806
246,-0.136747,-1.441671,-2.215733,-0.306886,-0.282379,0.834267,0.967400,-0.966865,-0.250128,-0.571810,...,-0.983685,0.640970,1.361407,0.624590,0.303295,-0.473015,-1.189466,-1.053966,0.831512,0.059324
247,-2.242234,-1.925303,-0.882575,0.840103,-0.921306,-2.452320,-0.539426,-0.161784,0.666097,1.453628,...,-1.218627,-0.599864,0.653637,-1.978012,0.524824,1.918404,-0.354402,0.719610,-1.246993,-1.005524
248,-0.355042,-0.215282,-0.882575,0.411593,0.011126,-1.269408,-1.115478,0.038999,0.464235,-0.179214,...,0.697950,0.385729,2.309332,-0.077654,0.248658,0.081710,0.197606,-2.199422,-0.534786,-0.180077


In [7]:
# Insert intercept variables
train_x.insert( 0, 'Ones', 1)
test_x.insert( 0, 'Ones', 1)
train_x

,Ones,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,1,-0.974717,-1.169956,-0.731456,0.535098,1.142759,0.540623,-0.423820,0.349920,-0.446382,...,0.201746,-0.331240,0.247523,-0.228612,0.439391,0.157961,0.328319,0.468780,1.054473,-0.971943
1,1,-0.737309,0.283037,1.682304,1.308350,1.821951,-0.074454,1.874460,-0.356719,0.675068,...,-0.758492,-0.709799,-1.130922,2.382589,0.940068,-1.396604,0.436464,-0.017203,-1.999355,-0.450893
2,1,0.198240,1.456971,-0.851523,-1.012478,-0.006887,0.032999,-1.439569,0.210541,-0.649364,...,-1.290767,0.756641,-0.977653,1.417600,0.660921,-0.255700,-0.316787,-0.141573,-0.640454,0.706032
3,1,-0.896252,-0.962236,-1.483945,0.287014,0.563168,0.697171,1.106720,0.085783,-0.081911,...,-1.348283,1.037693,0.581395,-0.620340,-0.555008,-0.116542,-0.898886,-1.116673,0.892800,3.604844
4,1,0.914488,-1.584348,0.446442,-0.255336,-0.245293,-0.780311,0.323170,-0.650096,-0.685251,...,-0.186249,0.679209,-0.993273,1.135747,-0.502358,-0.385326,1.648624,1.520175,-0.373112,-0.775872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,1,0.030243,-0.194300,-1.287284,0.786405,0.426482,1.550313,-0.238061,0.827510,1.055240,...,1.441769,1.359851,-0.325528,-0.248677,-0.976212,0.798469,1.863972,1.257849,1.132668,-0.514806
246,1,-0.136747,-1.441671,-2.215733,-0.306886,-0.282379,0.834267,0.967400,-0.966865,-0.250128,...,-0.983685,0.640970,1.361407,0.624590,0.303295,-0.473015,-1.189466,-1.053966,0.831512,0.059324
247,1,-2.242234,-1.925303,-0.882575,0.840103,-0.921306,-2.452320,-0.539426,-0.161784,0.666097,...,-1.218627,-0.599864,0.653637,-1.978012,0.524824,1.918404,-0.354402,0.719610,-1.246993,-1.005524
248,1,-0.355042,-0.215282,-0.882575,0.411593,0.011126,-1.269408,-1.115478,0.038999,0.464235,...,0.697950,0.385729,2.309332,-0.077654,0.248658,0.081710,0.197606,-2.199422,-0.534786,-0.180077


In [84]:

########### WARNING: VERY SLOW ###################

# GIVES THE SAME RESULT AS L1 WITH liblinear/saga

# Test saga solver with elasticnet regularization with various lambdas and l1 ratios
parameters = {'l1_ratio' : [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1], 'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 5, 10], 'solver':['saga'], 'penalty':['elasticnet']}
model=LogisticRegression(max_iter=5000, class_weight='balanced')
clf = GridSearchCV(model, parameters)
best_model = clf.fit(train_x,train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [ ]:
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best solver:', best_model.best_estimator_.get_params()['solver'])
print('Best C:', best_model.best_estimator_.get_params()['C'])
print('Best l1_ratio:', best_model.best_estimator_.get_params()['l1_ratio'])

In [80]:
# Test every solver with l2 regularization with various lambdas
parameters = {'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 5, 10], 'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'penalty':['l2']}
model=LogisticRegression(max_iter=5000, class_weight='balanced')
clf = GridSearchCV(model, parameters)
best_model = clf.fit(train_x,train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [85]:
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best solver:', best_model.best_estimator_.get_params()['solver'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

Best Penalty: elasticnet
Best solver: saga
Best C: 0.1


In [8]:
# Test liblinear and saga solvers with l1 regularization with various lambdas
parameters = {'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 5, 10], 'solver':['liblinear', 'saga'], 'penalty':['l1']}
model=LogisticRegression(max_iter=5000, class_weight='balanced')
clf = GridSearchCV(model, parameters)
best_model = clf.fit(train_x,train_y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` par

In [9]:
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best solver:', best_model.best_estimator_.get_params()['solver'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

Best Penalty: l1
Best solver: saga
Best C: 0.3


In [87]:
model=LogisticRegression(max_iter=5000, class_weight='balanced', C=0.2, penalty='l1', solver='liblinear')
selector = RFECV(model, step=0.1, cv=3, scoring='roc_auc', n_jobs=-1)
selector.fit(train_x, train_y)
print(selector.support_)
X_new = selector.transform(train_x)
X_test_new = selector.transform(test_x)
selector.fit(X_new, train_y)

[False False False False  True False False False False False False False
 False  True False  True  True  True False False False False False False
  True False False False False False  True False False  True False False
 False False False False False False False  True False False  True False
 False False  True False False False False False False False False False
 False False False  True False  True False  True False  True False False
 False  True False False False False False False  True False  True False
 False False False False False False  True  True False False False False
 False False False False False  True False False False False False False
  True False False  True False False  True False False  True False False
 False False False False False False False  True False  True  True False
  True  True  True False False False False False False False False False
 False  True False  True False False False False False False False False
 False False False False False False False False Fa

RFECV(cv=3,
      estimator=LogisticRegression(C=0.2, class_weight='balanced', dual=False,
                                   fit_intercept=True, intercept_scaling=1,
                                   l1_ratio=None, max_iter=5000,
                                   multi_class='warn', n_jobs=None,
                                   penalty='l1', random_state=None,
                                   solver='liblinear', tol=0.0001, verbose=0,
                                   warm_start=False),
      min_features_to_select=1, n_jobs=-1, scoring='roc_auc', step=0.1,
      verbose=0)

In [83]:
model=LogisticRegression(max_iter=5000, class_weight='balanced', C=0.1, penalty='l1', solver='liblinear')
model.fit(train_x, train_y)

selector = RFE(model, step=1)
selector.fit(train_x, train_y)
print(selector.support_)

[False False False False  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True False False False False False False
 False  True False False False False  True  True  True False  True False
 False False False False False False  True  True False False False False
 False False False False False  True False False False False False False
  True False False False False False False False False  True False False
 False False False False False False False  True  True  True False False
 False  True  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fa

In [88]:
# The best result seems to be with saga solver, l1 regularization and 0.1 regularization parameter
# Use the best model to predict
test_y=best_model.predict_proba(test_x)
print(test_y)

[[0.23699401 0.76300599]
 [0.4869367  0.5130633 ]
 [0.30969705 0.69030295]
 ...
 [0.61815862 0.38184138]
 [0.0623845  0.9376155 ]
 [0.71829035 0.28170965]]


In [89]:
# Get the values ready to write into file
test_y = pd.DataFrame(test_y[:,1])
indArr = []
for i in range(250, 20000):
    indArr.append(i)
test_y.insert(0, 'id', indArr, 1)
test_y.columns = ['id', 'target']

In [90]:
test_y.to_csv('/home/kito/Desktop/sem5/მანქანური სწავლება/blu/sample_submission.csv', index=False)